In [15]:
import csv
from collections import defaultdict, Counter
from typing import List, Dict, Tuple, Iterable, Set, Any, Optional


def _open_csv_autocodec(path: str, modes: Iterable[str] = ("utf-8-sig", "utf-8", "latin-1")):
    """
    Try opening CSV with several common encodings. Yields an open file object.
    """
    last_err = None
    for enc in modes:
        try:
            return open(path, newline="", encoding=enc), enc
        except UnicodeError as e:
            last_err = e
    # if all failed, re-raise last error
    raise last_err if last_err else OSError(f"Cannot open {path}")


def _read_csv_rows(path: str) -> Tuple[List[List[str]], str]:
    """
    Read all rows from CSV, returning (rows, encoding_used).
    Preserves all columns, does not strip header (bo nie mamy gwarancji, że jest).
    """
    f, enc = _open_csv_autocodec(path)
    with f:
        reader = csv.reader(f)
        rows: List[List[str]] = []
        for row in reader:
            # Normalize to strings, strip whitespace around cells
            if not row:
                continue
            cleaned = [cell.strip() for cell in row]
            # Skip if all cells are empty after stripping
            if not any(cleaned):
                continue
            rows.append(cleaned)
    return rows, enc


def _index_by_column(rows: List[List[str]], key_index: int) -> Dict[str, List[List[str]]]:
    """
    Map: word -> list of full rows where 'word' is rows[i][key_index].
    Skips rows that do not have the requested column.
    """
    idx: Dict[str, List[List[str]]] = defaultdict(list)
    for r in rows:
        if len(r) <= key_index:
            continue
        key = r[key_index].strip()
        if key == "":
            continue
        idx[key].append(r)
    return dict(idx)


def _max_len_of_rows(rows: Iterable[List[str]]) -> int:
    return max((len(r) for r in rows), default=0)


def _pad_row(row: List[str], target_len: int) -> List[str]:
    if len(row) >= target_len:
        return row
    return row + [""] * (target_len - len(row))


def _write_common_merged(
    common_words: Iterable[str],
    idx1: Dict[str, List[List[str]]],
    idx2: Dict[str, List[List[str]]],
    out_path: str,
    header: bool = True,
) -> int:
    """
    Zapisuje wszystkie dopasowania 'word' jako iloczyn kartezjański:
    [word] + row_from_file1 + row_from_file2

    Zwraca liczbę zapisanych wierszy.
    """
    # policz maksymalną liczbę kolumn per plik, by wyrównać długości
    max_c1 = _max_len_of_rows(r for rows in idx1.values() for r in rows)
    max_c2 = _max_len_of_rows(r for rows in idx2.values() for r in rows)

    count = 0
    with open(out_path, "w", newline="", encoding="utf-8-sig", errors="replace") as f:
        w = csv.writer(f)
        if header:
            w.writerow(["Word"] + [f"file1_col{i+1}" for i in range(max_c1)] + [f"file2_col{i+1}" for i in range(max_c2)])
        for word in sorted(common_words):
            rows1 = idx1.get(word, [])
            rows2 = idx2.get(word, [])
            for r1 in rows1:
                r1p = _pad_row(r1, max_c1)
                for r2 in rows2:
                    r2p = _pad_row(r2, max_c2)
                    w.writerow([word] + r1p + r2p)
                    count += 1
    return count


def _write_unique_rows(
    unique_words: Iterable[str],
    idx: Dict[str, List[List[str]]],
    out_path: str,
    file_label: str,
    header: bool = True,
) -> int:
    """
    Dla każdego 'word' wypisuje WSZYSTKIE wiersze z jednego źródła:
    [word] + row_from_that_file

    Zwraca liczbę zapisanych wierszy.
    """
    max_c = _max_len_of_rows(r for w in unique_words for r in idx.get(w, []))
    count = 0
    with open(out_path, "w", newline="", encoding="utf-8-sig", errors="replace") as f:
        w = csv.writer(f)
        if header:
            w.writerow(["Word"] + [f"{file_label}_col{i+1}" for i in range(max_c)])
        for word in sorted(unique_words):
            for r in idx.get(word, []):
                w.writerow([word] + _pad_row(r, max_c))
                count += 1
    return count


def compare_csv_words(
    file1: str,
    file2: str,
    *,
    file1_key_col: int = 1,  # 2nd column (0-based index)
    file2_key_col: int = 0,  # 1st column (0-based index)
    common_out: str = "common_merged.csv",
    unique1_out: str = "unique_file1.csv",
    unique2_out: str = "unique_file2.csv",
) -> Dict[str, Any]:
    """
    Główna funkcja:

    - file1: słowo brane z kolumny o indeksie file1_key_col (domyślnie 1 → 2. kolumna)
    - file2: słowo brane z kolumny o indeksie file2_key_col (domyślnie 0 → 1. kolumna)

    Zwraca słownik ze statystykami i zapisuje trzy pliki CSV:
      - common_merged.csv: dopasowania (iloczyn kartezjański wierszy) bok-w-bok
      - unique_file1.csv, unique_file2.csv: wiersze unikalne dla każdego pliku

    Statystyki obejmują zarówno unikalny słownik (SET), jak i realną liczbę wierszy (z duplikatami).
    """
    # Wczytaj oba pliki
    rows1, enc1 = _read_csv_rows(file1)
    rows2, enc2 = _read_csv_rows(file2)

    # Zindeksuj po odpowiednich kolumnach
    idx1 = _index_by_column(rows1, file1_key_col)
    idx2 = _index_by_column(rows2, file2_key_col)

    # Unikalne słowa
    keys1: Set[str] = set(idx1.keys())
    keys2: Set[str] = set(idx2.keys())

    common_words: Set[str] = keys1 & keys2
    unique1_words: Set[str] = keys1 - keys2
    unique2_words: Set[str] = keys2 - keys1

    # Liczby wierszy (z duplikatami) dla przejrzystości
    total_rows_file1 = sum(len(v) for v in idx1.values())
    total_rows_file2 = sum(len(v) for v in idx2.values())

    # Zapisz wyniki
    written_common = _write_common_merged(common_words, idx1, idx2, common_out)
    written_unique1 = _write_unique_rows(unique1_words, idx1, unique1_out, "file1")
    written_unique2 = _write_unique_rows(unique2_words, idx2, unique2_out, "file2")

    # Statystyki słownikowe (unikalne słowa)
    stats = {
        # Metadane wejścia
        "file1_path": file1,
        "file2_path": file2,
        "file1_encoding": enc1,
        "file2_encoding": enc2,

        # Unikalne słowa
        "unique_vocab_file1": len(keys1),
        "unique_vocab_file2": len(keys2),
        "common_vocab_count": len(common_words),
        "unique_to_file1_vocab_count": len(unique1_words),
        "unique_to_file2_vocab_count": len(unique2_words),
        "total_union_vocab": len(keys1 | keys2),

        # Wiersze (z duplikatami)
        "total_rows_file1": total_rows_file1,
        "total_rows_file2": total_rows_file2,

        # Ile wierszy zapisano w plikach wynikowych
        "rows_written_common": written_common,
        "rows_written_unique_file1": written_unique1,
        "rows_written_unique_file2": written_unique2,

        # Słowa (zestawy) – przydatne do dalszej obróbki
        "common_words": common_words,
        "unique_words_file1": unique1_words,
        "unique_words_file2": unique2_words,
    }
    return stats


def _print_summary(stats: Dict[str, Any]) -> None:
    """
    Ładne podsumowanie do konsoli.
    """
    print("=== INPUT ===")
    print(f"file1: {stats['file1_path']} (encoding: {stats['file1_encoding']})")
    print(f"file2: {stats['file2_path']} (encoding: {stats['file2_encoding']})")

    print("\n=== UNIQUE VOCAB (SETS) ===")
    print(f"Unique vocab in file1: {stats['unique_vocab_file1']}")
    print(f"Unique vocab in file2: {stats['unique_vocab_file2']}")
    print(f"Common vocab count   : {stats['common_vocab_count']} / {stats['total_union_vocab']} (union)")

    print("\n=== ROW COUNTS (WITH DUPLICATES) ===")
    print(f"Total rows used from file1 (keyed rows): {stats['total_rows_file1']}")
    print(f"Total rows used from file2 (keyed rows): {stats['total_rows_file2']}")

    print("\n=== OUTPUT FILES (ROWS WRITTEN) ===")
    print(f"common_merged.csv     : {stats['rows_written_common']} rows")
    print(f"unique_file1.csv      : {stats['rows_written_unique_file1']} rows")
    print(f"unique_file2.csv      : {stats['rows_written_unique_file2']} rows")


if __name__ == "__main__":
    # PRZYKŁAD UŻYCIA – zmień ścieżki na swoje
    file1 = r"NGSL_lists\NGSL-GR_rank.csv"           # słowo w 2. kolumnie (index 1)
    file2 = "4_NGSL_lists_combined.csv"      # słowo w 1. kolumnie (index 0)

    stats = compare_csv_words(
        file1,
        file2,
        file1_key_col=1,   # jeśli w file1 słowo jest w innej kolumnie, zmień indeks
        file2_key_col=0,   # jeśli w file2 słowo jest w innej kolumnie, zmień indeks
        common_out="common_merged.csv",
        unique1_out="unique_file1.csv",
        unique2_out="unique_file2.csv",
    )
    _print_summary(stats)


=== INPUT ===
file1: NGSL_lists\NGSL-GR_rank.csv (encoding: utf-8-sig)
file2: 4_NGSL_lists_combined.csv (encoding: utf-8-sig)

=== UNIQUE VOCAB (SETS) ===
Unique vocab in file1: 5051
Unique vocab in file2: 4832
Common vocab count   : 4031 / 5852 (union)

=== ROW COUNTS (WITH DUPLICATES) ===
Total rows used from file1 (keyed rows): 5051
Total rows used from file2 (keyed rows): 4832

=== OUTPUT FILES (ROWS WRITTEN) ===
common_merged.csv     : 4031 rows
unique_file1.csv      : 1020 rows
unique_file2.csv      : 801 rows
